## Computational Theory Assessment

In [1]:
import numpy as np

## Problem 1: Binary Words and Operations

In [2]:
def Parity(x, y, z):
    """Calculate parity using XOR: x ⊕ y ⊕ z"""
    return np.uint32(x ^ y ^ z)

def Ch(x, y, z):
    """Choose function: (x AND y) XOR (NOT x AND z)"""
    return np.uint32((x & y) ^ (~x & z))

def Maj(x, y, z):
    """Majority function: (x AND y) XOR (x AND z) XOR (y AND z)"""
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

def ROTR(x, n):
    """Rotate right by n positions"""
    return np.uint32((x >> n) | (x << (32 - n)))

def SHR(x, n):
    """Shift right by n positions"""
    return np.uint32(x >> n)

def Sigma0(x):
    """Σ₀ function: ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)"""
    return np.uint32(ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22))

def Sigma1(x):
    """Σ₁ function: ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)"""
    return np.uint32(ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25))

def sigma0(x):
    """σ₀ function: ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)"""
    return np.uint32(ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3))

def sigma1(x):
    """σ₁ function: ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)"""
    return np.uint32(ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10))

In [3]:
# Test Problem 1 functions
x = np.uint32(0xAAAAAAAA)
y = np.uint32(0x55555555)
z = np.uint32(0xFFFFFFFF)

print("Testing binary operations:")
print(f"Parity: {Parity(x, y, z):#010x}")
print(f"Ch: {Ch(x, y, z):#010x}")
print(f"Maj: {Maj(x, y, z):#010x}")
print(f"Sigma0: {Sigma0(x):#010x}")
print(f"Sigma1: {Sigma1(x):#010x}")
print(f"sigma0: {sigma0(x):#010x}")
print(f"sigma1: {sigma1(x):#010x}")

Testing binary operations:
Parity: 0x00000000
Ch: 0x55555555
Maj: 0xffffffff
Sigma0: 0x55555555
Sigma1: 0xaaaaaaaa
sigma0: 0xeaaaaaaa
sigma1: 0x002aaaaa


## Problem 2: Fractional Parts of Cube Roots

In [4]:
def primes(n):
    """Generate first n prime numbers using Sieve of Eratosthenes"""
    if n <= 0:
        return []
    
    if n < 6:
        limit = 15
    else:
        import math
        limit = int(n * (math.log(n) + math.log(math.log(n)))) + 100
    
    sieve = [True] * limit
    sieve[0] = sieve[1] = False
    
    for i in range(2, int(limit**0.5) + 1):
        if sieve[i]:
            for j in range(i*i, limit, i):
                sieve[j] = False
    
    result = []
    for i in range(limit):
        if sieve[i]:
            result.append(i)
            if len(result) == n:
                break
    
    return result

In [5]:
# Calculate K constants
first_64_primes = primes(64)
K_constants = []

for prime in first_64_primes:
    cube_root = prime ** (1/3)
    fractional_part = cube_root - int(cube_root)
    first_32_bits = int(fractional_part * (2**32))
    K_constants.append(np.uint32(first_32_bits))

print(f"Calculated {len(K_constants)} constants")
print(f"First 8: {[f'{k:#010x}' for k in K_constants[:8]]}")

Calculated 64 constants
First 8: ['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5']


In [6]:
# Verify against standard
STANDARD_K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

STANDARD_K = [np.uint32(k) for k in STANDARD_K]

all_match = all(calc == std for calc, std in zip(K_constants, STANDARD_K))
print(f"Constants match standard: {all_match}")

Constants match standard: True


## Problem 3: Padding

In [7]:
def block_parse(msg):
    """Pad message and yield 512-bit blocks"""
    msg_len_bits = len(msg) * 8
    padded = bytearray(msg)
    padded.append(0x80)
    
    while len(padded) % 64 != 56:
        padded.append(0x00)
    
    padded.extend(msg_len_bits.to_bytes(8, byteorder='big'))
    
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i+64])

In [8]:
# Test padding
test_messages = [b"", b"abc", b"a" * 55, b"a" * 56]

for msg in test_messages:
    blocks = list(block_parse(msg))
    print(f"Message length: {len(msg):3d} bytes -> {len(blocks)} block(s) of 64 bytes each")

Message length:   0 bytes -> 1 block(s) of 64 bytes each
Message length:   3 bytes -> 1 block(s) of 64 bytes each
Message length:  55 bytes -> 1 block(s) of 64 bytes each
Message length:  56 bytes -> 2 block(s) of 64 bytes each


## Problem 4: Hashes

## Problem 5: Passwords

## END